### Setting up Google Colab Environment

In [1]:
from google.colab import userdata
import os

os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')

In [2]:
!kaggle datasets download -d ruthgn/wine-quality-data-set-red-white-wine

Dataset URL: https://www.kaggle.com/datasets/ruthgn/wine-quality-data-set-red-white-wine
License(s): Attribution 4.0 International (CC BY 4.0)
  0% 0.00/98.0k [00:00<?, ?B/s]
100% 98.0k/98.0k [00:00<00:00, 231MB/s]


In [3]:
!unzip wine-quality-data-set-red-white-wine.zip

Archive:  wine-quality-data-set-red-white-wine.zip
  inflating: wine-quality-white-and-red.csv  


### Reading the CSV

In [4]:
import pandas as pd

wine_df = pd.read_csv("wine-quality-white-and-red.csv")
print(wine_df.describe())

       fixed acidity  volatile acidity  citric acid  residual sugar  \
count    6497.000000       6497.000000  6497.000000     6497.000000   
mean        7.215307          0.339666     0.318633        5.443235   
std         1.296434          0.164636     0.145318        4.757804   
min         3.800000          0.080000     0.000000        0.600000   
25%         6.400000          0.230000     0.250000        1.800000   
50%         7.000000          0.290000     0.310000        3.000000   
75%         7.700000          0.400000     0.390000        8.100000   
max        15.900000          1.580000     1.660000       65.800000   

         chlorides  free sulfur dioxide  total sulfur dioxide      density  \
count  6497.000000          6497.000000           6497.000000  6497.000000   
mean      0.056034            30.525319            115.744574     0.994697   
std       0.035034            17.749400             56.521855     0.002999   
min       0.009000             1.000000         

### Adding Dataset Describers

In [53]:
precision = 4

def prec(n, p=precision):
  return f"{n:.{p}f}"

std_describers = {
    'mean': lambda x: prec(x.mean()),
    'median': lambda x: prec(x.median()),
    'std': lambda x: prec(x.std()),
    'variation': lambda x: prec(x.var(), 8),
    'range of values': lambda x: f"{x.min()} ... {x.max()}"
}

categ_describers = {
    'mode': lambda x: x.mode().iloc[0]
}

def describe_column(col: pd.Series):
  for desc, func in std_describers.items():
    print(f"{desc}: {func(col)}")


def describe_cat_column(col: pd.Series):
  for desc, func in categ_describers.items():
      print(f"{desc}: {func(col)}")

In [55]:
cat_columns = ["type"]

for column in wine_df.columns:
  print(f"{column}")
  if column in cat_columns:
    describe_cat_column(wine_df[column])
  else:
    describe_column(wine_df[column])
  print("\n")

type
mode: white


fixed acidity
mean: 7.2153
median: 7.0000
std: 1.2964
variation: 1.68074049
range of values: 3.8 ... 15.9


volatile acidity
mean: 0.3397
median: 0.2900
std: 0.1646
variation: 0.02710517
range of values: 0.08 ... 1.58


citric acid
mean: 0.3186
median: 0.3100
std: 0.1453
variation: 0.02111728
range of values: 0.0 ... 1.66


residual sugar
mean: 5.4432
median: 3.0000
std: 4.7578
variation: 22.63669646
range of values: 0.6 ... 65.8


chlorides
mean: 0.0560
median: 0.0470
std: 0.0350
variation: 0.00122735
range of values: 0.009 ... 0.611


free sulfur dioxide
mean: 30.5253
median: 29.0000
std: 17.7494
variation: 315.04119227
range of values: 1.0 ... 289.0


total sulfur dioxide
mean: 115.7446
median: 118.0000
std: 56.5219
variation: 3194.72003868
range of values: 6.0 ... 440.0


density
mean: 0.9947
median: 0.9949
std: 0.0030
variation: 0.00000899
range of values: 0.98711 ... 1.03898


pH
mean: 3.2185
median: 3.2100
std: 0.1608
variation: 0.02585252
range of values: 2.7

### Performing Correlation Analysis

In [91]:
corr_methods = ["pearson", "kendall", "spearman"]

corr_list = []

def describe_correlation(col: pd.Series):
  acc = 0
  temp_df = pd.DataFrame(col).join(wine_df["quality"])
  for corr_method in corr_methods:
    corr_matrix = temp_df.corr(method=corr_method)
    corr = corr_matrix.iloc[0, 1]
    print(f"{corr_method}: {prec(corr)}")
    acc += corr

  avg = acc / len(corr_methods)
  print(f"avg: {prec(avg)}")
  print("\n")

  corr_list.append((col.name, avg))

In [92]:
ignore_cols = ["quality", "type"]

for column in wine_df.columns:
  if column in ignore_cols:
    # do nothing
    pass
  else:
    print(f"{column}")
    describe_correlation(wine_df[column])

fixed acidity
pearson: -0.0767
kendall: -0.0760
spearman: -0.0982
avg: -0.0836


volatile acidity
pearson: -0.2657
kendall: -0.1991
spearman: -0.2578
avg: -0.2409


citric acid
pearson: 0.0855
kendall: 0.0822
spearman: 0.1057
avg: 0.0911


residual sugar
pearson: -0.0370
kendall: -0.0131
spearman: -0.0169
avg: -0.0223


chlorides
pearson: -0.2007
kendall: -0.2289
spearman: -0.2951
avg: -0.2415


free sulfur dioxide
pearson: 0.0555
kendall: 0.0667
spearman: 0.0869
avg: 0.0697


total sulfur dioxide
pearson: -0.0414
kendall: -0.0423
spearman: -0.0548
avg: -0.0461


density
pearson: -0.3059
kendall: -0.2480
spearman: -0.3228
avg: -0.2922


pH
pearson: 0.0195
kendall: 0.0252
spearman: 0.0325
avg: 0.0258


sulphates
pearson: 0.0385
kendall: 0.0237
spearman: 0.0298
avg: 0.0307


alcohol
pearson: 0.4443
kendall: 0.3524
spearman: 0.4469
avg: 0.4146




In [100]:
sorted_corr = sorted(corr_list, key=lambda tup: abs(tup[1]), reverse=True)

print("Most correlated features (by absolute value):")
for col, num in sorted_corr:
  print(f"{col}: {num}")

Most correlated features (by absolute value):
alcohol: 0.414557993915356
density: -0.2922138409852008
chlorides: -0.2415306821642226
volatile acidity: -0.2408688872284482
citric acid: 0.09113402112386733
fixed acidity: -0.0836292658322374
free sulfur dioxide: 0.06968050656192347
total sulfur dioxide: -0.04614853225250134
sulphates: 0.030665022500617287
pH: 0.02575549621145906
residual sugar: -0.022322853947869297


### Get Stats for Best Wines

In [108]:
best_wine_df = wine_df[wine_df["quality"] == wine_df["quality"].max()]
print(best_wine_df.describe())

       fixed acidity  volatile acidity  citric acid  residual sugar  \
count          5.000             5.000        5.000           5.000   
mean           7.420             0.298        0.386           4.120   
std            0.983             0.058        0.082           3.759   
min            6.600             0.240        0.290           1.600   
25%            6.900             0.260        0.340           2.000   
50%            7.100             0.270        0.360           2.200   
75%            7.400             0.360        0.450           4.200   
max            9.100             0.360        0.490          10.600   

       chlorides  free sulfur dioxide  total sulfur dioxide  density     pH  \
count      5.000                5.000                 5.000    5.000  5.000   
mean       0.027               33.400               116.000    0.991  3.308   
std        0.007               13.428                19.824    0.003  0.083   
min        0.018               24.000       

### Get Stats for Worst Wines

In [112]:
worst_wine_df = wine_df[wine_df["quality"] == wine_df["quality"].min()]
print(worst_wine_df.describe())

       fixed acidity  volatile acidity  citric acid  residual sugar  \
count         30.000            30.000       30.000          30.000   
mean           7.853             0.517        0.281           5.140   
std            1.748             0.342        0.174           4.731   
min            4.200             0.170        0.000           0.700   
25%            6.800             0.253        0.212           1.650   
50%            7.450             0.415        0.330           3.150   
75%            8.575             0.633        0.395           7.800   
max           11.800             1.580        0.660          16.200   

       chlorides  free sulfur dioxide  total sulfur dioxide  density      pH  \
count     30.000               30.000                30.000   30.000  30.000   
mean       0.077               39.217               122.033    0.996   3.258   
std        0.062               59.990               112.142    0.003   0.213   
min        0.022                3.000   

### Compare Best to Worst

In [114]:
ignore_columns = ["type", "quality"]

for column, _ in sorted_corr:
  print(f"{column}\n")
  if column in ignore_columns:
    print("Best:")
    describe_cat_column(best_wine_df[column])
    print("\nWorst:")
    describe_cat_column(worst_wine_df[column])
  else:
    print("Best:")
    describe_column(best_wine_df[column])
    print("\nWorst:")
    describe_column(worst_wine_df[column])
  print("\n")

alcohol

Best:
mean: 12.1800
median: 12.5000
std: 1.0134
variation: 1.02700000
range of values: 10.4 ... 12.9

Worst:
mean: 10.2150
median: 10.1500
std: 1.1065
variation: 1.22433621
range of values: 8.0 ... 12.6


density

Best:
mean: 0.9915
median: 0.9903
std: 0.0031
variation: 0.00000972
range of values: 0.98965 ... 0.997

Worst:
mean: 0.9957
median: 0.9959
std: 0.0028
variation: 0.00000802
range of values: 0.9911 ... 1.0008


chlorides

Best:
mean: 0.0274
median: 0.0310
std: 0.0074
variation: 0.00005530
range of values: 0.018 ... 0.035

Worst:
mean: 0.0770
median: 0.0550
std: 0.0620
variation: 0.00384569
range of values: 0.022 ... 0.267


volatile acidity

Best:
mean: 0.2980
median: 0.2700
std: 0.0576
variation: 0.00332000
range of values: 0.24 ... 0.36

Worst:
mean: 0.5170
median: 0.4150
std: 0.3419
variation: 0.11690448
range of values: 0.17 ... 1.58


citric acid

Best:
mean: 0.3860
median: 0.3600
std: 0.0820
variation: 0.00673000
range of values: 0.29 ... 0.49

Worst:
mean: 0.28